In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
    
    # this is a static function, similar to C++
    # does not binded with a class instance
    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))
        
    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
        
    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_,
                                 self.width_, axis=1)
    

In [2]:
#Data wrangling

counts = pd.read_csv('data/FremontBridge.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv('data/BicycleWeather.csv', index_col='DATE', parse_dates=True)

In [3]:
daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)
    
# add an indicator about holiday
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
# replace missing data with 0
daily['holiday'].fillna(0, inplace=True)  

def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))
#daily[['daylight_hrs']].plot()
#plt.ylim(8, 17)


<ipython-input-3-d8c1da814533>:19: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  days = (date - pd.datetime(2000, 12, 21)).days


In [4]:
# temperatures are in 1/10 deg C; convert to C
weather['TMIN'] /= 10
weather['TMAX'] /= 10
weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# precip is in 1/10 mm; convert to inches
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)

daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])
# how='left' means calling frame’s index 
# daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']], how='left', lsuffix='_left', rsuffix='_right')


In [5]:
# number of years passed
daily['annual'] = (daily.index - daily.index[0]).days / 365.

In [6]:
from sklearn.model_selection import cross_val_score
# Drop any rows with null values
daily.dropna(axis=0, how='any', inplace=True)

column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'holiday',
                'daylight_hrs', 'PRCP', 'dry day', 'Temp (C)', 'annual']
X = daily[column_names]
y = daily['Total']

lrmodel = LinearRegression(fit_intercept=False)
lrmodel.fit(X, y)
daily['predicted'] = lrmodel.predict(X)
cross_val_score(lrmodel,X,y, cv=10)
np.mean(cross_val_score(lrmodel,X,y,cv=10))

#model.fit(X, y)
#daily['predicted'] = model.predict(X)
#daily[['Total', 'predicted']].plot(alpha=0.5);

0.7691752923789886

In [7]:
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

param_grid = {'alpha':sp_rand()}
model= Lasso()
rand_search = RandomizedSearchCV(estimator=model, param_distributions= param_grid, n_iter=100, cv=10)
rand_search.fit(X,y)

print('alpha: %f' % rand_search.best_estimator_.alpha)
print('best_score: %f' % rand_search.best_score_)

lasso= LassoCV(cv=10).fit(X,y)
print('CV Score: %f' % lasso.score(X,y))

#model = Lasso(alpha=0.9948557117518596)
#model.fit(X,y)
#daily['predicted'] = model.predict(X)
#daily[['Total', 'predicted']].plot(alpha=0.9948557117518596);

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.623e+05, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.301e+05, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.260e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.484e+05, tolerance: 1.458e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.683e+05, tolerance: 1.521e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.738e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.820e+05, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.487e+05, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.429e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.687e+05, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.363e+05, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.318e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.512e+05, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.194e+05, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.158e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.821e+05, tolerance: 1.562e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.802e+05, tolerance: 1.505e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.583e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.708e+06, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.656e+06, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.627e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.746e+05, tolerance: 1.476e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.419e+05, tolerance: 1.544e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.370e+

/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.766e+05, tolerance: 1.505e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.556e+05, tolerance: 1.407e+05
  model = cd_fast.enet_coordinate_descent(
/home/naomi/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.446e+

alpha: 0.983986
best_score: 0.769895
CV Score: 0.866028


In [8]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

param_grid = {'alpha':sp_rand()}
model= Ridge()
rand_search = RandomizedSearchCV(estimator=model, param_distributions= param_grid, n_iter=100, cv=10)
rand_search.fit(X,y)

print('alpha: %f' % rand_search.best_estimator_.alpha)
print('best_score: %f' % rand_search.best_score_)


ridge= RidgeCV(cv=10).fit(X,y)
print('CV Score: %f' % ridge.score(X,y))


alpha: 0.999284
best_score: 0.769976
CV Score: 0.867483


In [9]:
#Lasso Model Performance

from sklearn.model_selection import RepeatedKFold

lmodel = Lasso(alpha=0.964794)
lmodel.fit(X,y)

cv= RepeatedKFold(n_splits=10, n_repeats =10, random_state=1)

scores= cross_val_score(lmodel,X,y, cv=cv)
                     
#print(cross_val_score(rmodel,X,y, cv=10))

np.mean(cross_val_score(lmodel,X,y,cv=10))


0.7698824103462483

In [10]:
#Ridge Model Performance

rmodel = Ridge(alpha=0.976371)
rmodel.fit(X,y)

cv= RepeatedKFold(n_splits=10, n_repeats =10, random_state=1)


print(cross_val_score(rmodel,X,y, cv=10))

np.mean(cross_val_score(rmodel,X,y,cv=10))

[0.76659344 0.75097497 0.77039215 0.84568192 0.73942685 0.75901346
 0.82076614 0.75114736 0.77730569 0.7183024 ]


0.7699604374847081